In [2]:
import numpy as np
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Grid, LinearAxis, Plot, Rect, Text, HoverTool
import pandas as pd
from code.classes.schedule import Schedule

ModuleNotFoundError: No module named 'code.classes'; 'code' is not a package

In [ ]:
output_notebook()

In [ ]:
def visualize_schedule(schedule):
    print(schedule.get_dataframe())
    return

In [3]:
def time_ticker_func(tick_value):
    """
    Takes the tick value and returns the corresponding time.
    """
    
    if tick_value == 25:
        return "19:00"
    elif tick_value % 5 == 1:
        return "11:00"
    elif tick_value % 5 == 2:
        return "13:00"
    elif tick_value % 5 == 3:
        return "15:00"
    elif tick_value % 5 == 4:
        return "17:00"
    else:
        return "09:00"
    
def day_ticker_func(tick_value):
    """
    Takes the tick_value and returns the corresponding day.
    """
    
    # TODO: magic numbers weg
    tick_to_day = {2.5: "Monday", 7.5: "Tuesday", 12.5: "Wednesday", 17.5: "Thursday", 22.5: "Friday"}
    
    return tick_to_day[tick_value]
    
def color_pick(day_index):
    """
    Takes the day index (Monday = 0, Tuesday = 1, Wednesday = 2, Thursday = 3, Friday = 4) and
    returns one of two colors, in order to distinguish the days from each other in the schedule.
    """
    
    if day_index % 2 == 0:
        return "lavenderblush"
    
    return "pink"

In [4]:
# TODO: werken met schedule object en zijn df attribuut

output_file("../output_data/schedule.html")

# read in the schedule dataframe
schedule_df = pd.read_csv("../output_data/schedule_student_numbers.csv").set_index("Time")

# read in the room dataframe
rooms_df = pd.read_csv("../input_data/rooms.csv").rename(columns={"Zaalnummber":"room", "Max. capaciteit" : "capacity"})
rooms_df["capacity"] = rooms_df["capacity"].apply(str)

# convert the room dataframe to a dictionary -> {index (int): {"room" : room number (str), "capacity": room capacity (str)}}
room_dict = rooms_df.to_dict('index')

# create the plot on which we design the schedule
schedule = Plot(width=2000, height=2000)

# width and height of each lesson's visual representation (rectangle)
w = np.full(35, .95)
h = np.full(35, .8)

number_of_rooms = 7

number_of_days = 5
time_slots_per_day = 5
total_slots = number_of_days * time_slots_per_day

# keep track of the x and y coordinates of each day's lessons (rectangles)
x_values = np.array([])
y_values = np.array([])

# keep track of the names of each day's lessons (rectangles)
courses = []

# loop over the time slots
for y in range(total_slots):
    
    # loop over the rooms
    for x in range(number_of_rooms):
        
        x_values = np.append(x_values, x)
        y_values = np.append(y_values, y + .5)

        # append the course name
        course_info = schedule_df.iloc[y][x]
        course_name = str(course_info.split("|")[0])
        courses.append(course_name if len(course_name) > 1 else "")
    
    # one day has passed (e.g. at index 4, 9 etc.)
    if y % 5 == 4: 
        
        # add one day's rectangles to schedule
        rect_source = ColumnDataSource(dict(x=x_values, y=y_values, w=w, h=h))
        rectangles = Rect(x="x", y="y", width="w", height="h", fill_color=color_pick(y + 1))
        schedule.add_glyph(rect_source, rectangles)

        # create arrays for the text's x and y coordinates to make sure the text aligns nicely with rectangles
        text_x_values = x_values - .45
        text_y_values = y_values + .1

        # add text
        text_source = ColumnDataSource(dict(x=text_x_values, y=text_y_values, text=courses))
        my_text = Text(x="x", y="y", text="text")
        my_text.text_font_size = {'value': '11px'}
        schedule.add_glyph(text_source, my_text)
        
        # empty the arrays
        x_values = np.array([])
        y_values = np.array([])
        courses = []
    
# add x and y axis - ???
xaxis = LinearAxis(axis_label="Rooms")
schedule.add_layout(xaxis, 'above')
yaxis = LinearAxis(axis_label='Time slots')
schedule.add_layout(yaxis, 'left')
yaxis2 = LinearAxis(axis_label='Days')
schedule.add_layout(yaxis2, 'left')

# ???
schedule.add_layout(Grid(dimension=0, ticker=xaxis.ticker))
schedule.add_layout(Grid(dimension=1, ticker=yaxis.ticker))
schedule.add_layout(Grid(dimension=1, ticker=yaxis2.ticker))

# create room tick labels
schedule.xaxis.major_label_overrides = {num : (room_dict[num]["room"] + " | " + room_dict[num]["capacity"]) for num in range(number_of_rooms)}
schedule.xaxis.minor_tick_line_color = None

# make sure we start at 0 in the upper left corner
schedule.y_range.flipped = True 

# create time tick labels (0 up to and including 25, to get a "19:00" tick)
schedule.yaxis[0].ticker = np.arange(total_slots + 1)
schedule.yaxis[0].major_label_overrides = {num : time_ticker_func(num) for num in range(total_slots + 1)}

# create day tick labels (starting at 2.5 with steps of 5, to get ticks halfway through each day)
schedule.yaxis[1].ticker = np.arange((time_slots_per_day / 2), total_slots + (time_slots_per_day / 2), number_of_days)
schedule.yaxis[1].major_label_overrides = {num : day_ticker_func(num) for num in np.arange((time_slots_per_day / 2), total_slots + (time_slots_per_day / 2), number_of_days)}

# TODO: de lesson waarden achterhalen -> df maken met type, group, students en malus points?
# laad de rooms in en sla ze op in een dict. Deze omzetten in df.
# lekker hoveren 
hover = HoverTool()
hover.tooltips = """
    <div>
        <div><strong>Type: </strong>Lab</div>
        <div><strong>Group: </strong>3</div>
        <div><strong>Students: </strong>18</div>
        <div><strong>Malus points: </strong>454</div>
    </div>
"""

schedule.add_tools(hover)

# show the results
show(schedule)

FileNotFoundError: [Errno 2] No such file or directory: '../output_data/schedule_student_numbers.csv'

### TO DO
- lesson info achterhalen uit bestaande code -> implementeren in hovertool
- ook nog welke studenten in hovertool?
- (selection tool voor individuele roosters)
- rooms oplopend in capaciteit